# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

In [1]:
import numpy as np
import nltk
import pandas as pd
import pymorphy2
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

## Задачи для совместного разбора

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
with open('data/litw-win.txt', encoding='cp1251') as f:
    words = [line.strip().split()[-1] for line in f]

words[-5:]

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [ ]:
words = set(words)
for word in text.split():
    if word not in words:
        print(word)

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
stemmer = SnowballStemmer('russian')
stemmer.stem('попреблагорассмотрительствующемуся')

In [ ]:
morph = pymorphy2.MorphAnalyzer()
morph.parse('попреблагорассмотрительствующемуся')[0].normalized.word

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
text = 'Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.'
sent_list = nltk.sent_tokenize(text)
sent_list

In [ ]:
cv = CountVectorizer()
cv.fit(sent_list)
sent_cv = cv.transform(sent_list).toarray()
sent_cv

In [ ]:
cv.vocabulary_

In [ ]:
set(nltk.word_tokenize(text)) - cv.vocabulary_.keys()

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [3]:
descriptions_df = pd.read_csv('data/preprocessed_descriptions.csv', sep=',')
descriptions_df = descriptions_df.rename(columns={'preprocessed_descriptions': 'description'})
descriptions_df.info()
descriptions_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         30000 non-null  object
 1   description  29369 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [4]:
description = descriptions_df['description'].dropna()
words = {word for text in description for word in nltk.word_tokenize(text)}
len(words)

30823

In [ ]:
%%timeit

{word for text in description for word in nltk.word_tokenize(text)}


In [ ]:
%%timeit

words = set()
{word for text in description.array for word in nltk.word_tokenize(text)}

In [5]:
dasha_words = set(np.concatenate(description.apply(nltk.word_tokenize).values))
len(dasha_words)

30823

In [10]:
dct = {i: i ** 3 for i in range(10)}
list(dct.values())[:5]

[0, 1, 8, 27, 64]

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).